In [8]:
from dataclasses import dataclass
@dataclass
class Schema:
    datetime_beginning_utc:str="datetime64[ns]"
    datetime_beginning_ept:str="datetime64[ns]"
    nerc_region:str="string"
    mkt_region:str="string"
    zone:str="string"
    load_area:str="string"
    mw:float="float64"
    is_verified="boolean"

In [9]:
import pandas as pd
from usagrid import s3

fn = s3.list_files_in_folder("usagrid",folder_name="bronze/PJM/load_metered")[0]
df = s3.read_pyarrow_from_s3(bucket_name="usagrid",key=fn).to_pandas()

In [10]:
df.head()

,datetime_beginning_utc,datetime_beginning_ept,nerc_region,mkt_region,zone,load_area,mw,is_verified
0,1/1/2020 4:00:00 AM,12/31/2019 11:00:00 PM,RFC,MIDATL,AE,AECO,953.387,True
1,1/1/2020 4:00:00 AM,12/31/2019 11:00:00 PM,RFC,WEST,AEP,AEPAPT,4414.586,True
2,1/1/2020 4:00:00 AM,12/31/2019 11:00:00 PM,RFC,WEST,AEP,AEPIMP,2638.794,True
3,1/1/2020 4:00:00 AM,12/31/2019 11:00:00 PM,RFC,WEST,AEP,AEPKPT,687.505,True
4,1/1/2020 4:00:00 AM,12/31/2019 11:00:00 PM,RFC,WEST,AEP,AEPOPT,5838.740,True


In [11]:
schema = Schema().__dict__
for k,v in schema.items():
    
    print(k,v)

    df[k] = df[k].astype(v)

datetime_beginning_utc datetime64[ns]
datetime_beginning_ept datetime64[ns]
nerc_region string
mkt_region string
zone string
load_area string
mw float64


In [13]:
df_oriented = df.sort_values("datetime_beginning_utc")\
                .reset_index(drop=True).set_index("datetime_beginning_utc")

df_oriented["2023":]

,datetime_beginning_ept,nerc_region,mkt_region,zone,load_area,mw,is_verified
datetime_beginning_utc,,,,,,,
2023-01-01 00:00:00,2022-12-31 19:00:00,RFC,WEST,DUQ,DUQ,1388.314,False
2023-01-01 00:00:00,2022-12-31 19:00:00,RFC,MIDATL,DPL,DPLCO,1998.345,True
2023-01-01 00:00:00,2022-12-31 19:00:00,SERC,SOUTH,DOM,DOM,12480.694,True
2023-01-01 00:00:00,2022-12-31 19:00:00,RFC,WEST,DEOK,DEOK,2708.360,True
2023-01-01 00:00:00,2022-12-31 19:00:00,RFC,WEST,DAY,DAY,1706.000,False
...,...,...,...,...,...,...,...
2024-06-18 03:00:00,2024-06-17 23:00:00,RFC,MIDATL,RECO,RECO,209.887,False
2024-06-18 03:00:00,2024-06-17 23:00:00,RFC,MIDATL,PEP,SMECO,427.960,False
2024-06-18 03:00:00,2024-06-17 23:00:00,RFC,MIDATL,AE,VMEU,96.608,False


In [14]:
s3.write_pyarrow_to_s3(bucket_name="usagrid",object_key="silver/PJM/PJM_load_metered.arrow",data=df_oriented)